In [1]:
# code adapted from https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33#:~:text=The%20Keras%20ResNet%20got%20to,to%20do%20with%20weight%20initializations.
# import plaidml
# import plaidml.keras
# plaidml.install_backend()
# import os
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
    
import random
random.seed(10)

from PIL import Image
from glob import glob

# Clean Script
from glob import glob
import tensorflow as tf
import os

from tensorflow.keras.utils import load_img

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

path_to_directory = 'imagedata/'


# Collect paths to images based on label
patientPaths= glob(os.path.join(path_to_directory, "*", ""))

# print(patientPaths)
nonCancerTrPaths = []
nonCancerTestPaths = []
cancerousTrPaths = []
cancerousTestPaths = [] 
nonCancerValPaths = []
cancerousValPaths = []

for i in range(0,168):
    nonCancerTrPaths.extend(glob(os.path.join(patientPaths[i], "0/", "*")))
    cancerousTrPaths.extend(glob(os.path.join(patientPaths[i], "1/", "*")))

for i in range(168, 224):
    nonCancerValPaths.extend(glob(os.path.join(patientPaths[i], "0/", "*")))
    cancerousValPaths.extend(glob(os.path.join(patientPaths[i], "1/", "*")))

for i in range(224,280):
    nonCancerTestPaths.extend(glob(os.path.join(patientPaths[i], "0/", "*")))
    cancerousTestPaths.extend(glob(os.path.join(patientPaths[i], "1/", "*")))


In [2]:
# Turn filepaths into image arrays to train a model
def paths_to_image(paths, label, num_samples):
    images = []
    labels = []
    for path in paths[0:num_samples]:
        image = load_img(path)
        image = image.resize([50, 50])
        imgArray = tf.keras.utils.img_to_array(image)
        images.append(imgArray)
        labels.append(label)
    return [images[0: num_samples], labels[0: num_samples]]

nonCancerTrImages = paths_to_image(nonCancerTrPaths, 0, len(nonCancerTrPaths))
cancerTrImages = paths_to_image(cancerousTrPaths, 1, len(cancerousTrPaths))

nonCancerValImages = paths_to_image(nonCancerValPaths, 0, len(nonCancerValPaths))
cancerValImages = paths_to_image(cancerousValPaths, 1, len(cancerousValPaths))

nonCancerTestImages = paths_to_image(nonCancerTestPaths, 0, len(nonCancerTestPaths))
cancerTestImages = paths_to_image(cancerousTestPaths, 1, len(cancerousTestPaths))


In [3]:
x_train = nonCancerTrImages[0] + cancerTrImages[0]
y_train = nonCancerTrImages[1] + cancerTrImages[1]

x_val = nonCancerValImages[0] + cancerValImages[0]
y_val = nonCancerValImages[1] + cancerValImages[1]

x_test = nonCancerTestImages[0]+ cancerTestImages[0]
y_test = nonCancerTestImages[1]+ cancerTestImages[1]

# # split images into train, val, test
# allImages = nonCancerImages[0] + cancerImages[0]
# allLabels = nonCancerImages[1] + cancerImages[1]

# x_train, x_toSplit, y_train, y_toSplit = train_test_split(allImages, allLabels, test_size=.5, stratify= allLabels, random_state = 10)
# x_val, x_test, y_val, y_test = train_test_split(x_toSplit, y_toSplit, test_size=.5, stratify=y_toSplit, random_state = 10)


#function which rotates image a certain number of degrees = to rotation
def rotate_image(X, y, rotation, num_to_rotate = None):
    rotatedImages = []
    labels = []
    cancerX = X[y==1]
    if num_to_rotate is not None and num_to_rotate < len(cancerX):
        cancerX = cancerX[0: num_to_rotate]
    for img in cancerX:
        image = tf.keras.utils.array_to_img(img)
        image = image.resize([50, 50])
        rotateimg = image.rotate(rotation)
        imgArray = tf.keras.utils.img_to_array(rotateimg)
        rotatedImages.append(imgArray)
        labels.append(1)
    return [rotatedImages, labels]


train90 = rotate_image(np.array(cancerTrImages[0]), np.array(cancerTrImages[1]), 90, 23994)
train180 = rotate_image(np.array(cancerTrImages[0]), np.array(cancerTrImages[1]), 180, 23995)
train270 = rotate_image(np.array(cancerTrImages[0]), np.array(cancerTrImages[1]), 270, 23995)


In [4]:
print("Number of cancer images in training set", str((len(train180[0]) + len(train90[0])) + len(train270[0]) + sum(y_train)))
print("Number of benign images in training set", str(len(y_train) - sum(y_train)))


Number of cancer images in training set 121006
Number of benign images in training set 121006


In [5]:
x_train = np.array(x_train)
train90[0] = np.array(train90[0])
train180[0] = np.array(train180[0])
y_train = np.array(y_train)
train90[1] = np.array(train90[1])
train180[1] = np.array(train180[1])

training_x = np.concatenate((x_train, train90[0], train180[0],train270[0]), axis=0)
training_y = np.concatenate((y_train, train90[1], train180[1],train270[1]))


# Shuffle order
zipper = list(zip(training_x, training_y))
random.shuffle(zipper)
training_x, training_y = zip(*zipper)

datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
datagen.fit(training_x)

# Because of computing power problems we subset to the first 20000 images in training
# training_x = training_x[0:20000]
# training_y = training_y[0:20000]
train_x_param = training_x[0:10000]
train_y_param = training_y[0:10000]

train_param_iterator = datagen.flow(np.array(train_x_param), np.array(train_y_param))

train_iterator = datagen.flow(np.array(training_x), np.array(training_y))
val_iterator = datagen.flow(np.array(x_val), np.array(y_val))


In [7]:
# #Learning rates above .01 are not useful
import keras.backend as K
import tensorflow_addons as tfa

learning_rates = [.0001, .0005, .001, .005, .01, .05, .1]

for lr in learning_rates:
    model = VGG16(weights=None, include_top=True, input_shape= (50, 50,3), classes=2, classifier_activation='softmax')
    adam = Adam(learning_rate=lr)
    model.compile(optimizer= adam, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy', tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2)])

    print("FITTING")
    history = model.fit(train_param_iterator, epochs=10, validation_data=val_iterator)
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')
    plt.title("Learning Rate: " + str(lr))
    plt.show()

# The model outputs a value between 0 and 1, which can be thought of as a probability of being a cancer or not
# model.predict to find the pseudo probability which is a measure of confidence
# we can output the score and the prediction
# activation map to find the feature


# LR = .0001 achieves the highest validation accuracy
# Increase number of epochs to see if more epochs further increases val error
# LR = .0001
# model = VGG16(weights=None, include_top=True, input_shape= (50, 50,3), classes=2, classifier_activation='softmax')
# adam = Adam(learning_rate=LR)
# model.compile(optimizer= adam, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy', tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2)])

# print("FITTING")
# history = model.fit(train_iterator, epochs=15, validation_data=val_iterator)
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')
# plt.title("Learning Rate: " + str(LR))
# plt.show()

# model.save("FullTrain2023")



FITTING
Epoch 1/10
313/313 [==============================] - 343s 1s/step - loss: 0.5538 - accuracy: 0.7325 - MatthewsCorrelationCoefficient: 0.0000e+00 - val_loss: 0.4396 - val_accuracy: 0.8080 - val_MatthewsCorrelationCoefficient: 0.0000e+00
Epoch 2/10
 10/313 [..............................] - ETA: 2:53 - loss: 0.5256 - accuracy: 0.7625 - MatthewsCorrelationCoefficient: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# LR = .0001 achieves the highest validation accuracy
# Increase number of epochs to see if more epochs further increases val error
# model = VGG16(weights=None, include_top=True, input_shape= (50, 50,3), classes=2, classifier_activation='softmax')
# adam = Adam(learning_rate=.0001)
# model.compile(optimizer= adam, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

# print("FITTING")
# history = model.fit(np.array(x_train), np.array(y_train), epochs=15, validation_data=(np.array(x_test), np.array(y_test)))
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')
# plt.title("Learning Rate: " + str(lr))
# plt.show()


In [ ]:



# model = tf.keras.models.load_model('April1AB20k2023')

In [ ]:
model.summary()

In [ ]:
test_iterator = datagen.flow(np.array(x_test), np.array(y_test))

model.evaluate(test_iterator)

In [ ]:
a = np.array(y_test)
a[a==1].shape
 

acc = (49684*0.4283320903778076+0.8424640893936157*19697) /(49684+19697)
acc

In [ ]:
y_prediction = model.predict(np.array(x_test))
# ypred2 = model.predict_classes(np.array(x_test))


In [ ]:
prediction = np.zeros(y_prediction.shape[0])
mask = (y_prediction[:,0]<0.5)
prediction[mask] = 1
print(prediction)

equal = np.zeros(y_prediction.shape[0])
equal[prediction == np.array(y_test)] = 1

accuracy = np.sum(equal)/y_prediction.shape[0]
accuracy

In [ ]:
y_test[np.array(y_test) == 0]

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
result = confusion_matrix(y_test, prediction , normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=result )

disp.plot()


In [ ]:
prediction = np.zeros(y_prediction.shape[0])
mask = (y_prediction[:,1]<0.8)
prediction[mask] = 1
print(prediction)

equal = np.zeros(y_prediction.shape[0])
equal[prediction == np.array(y_test)] = 1

accuracy = np.sum(equal)/y_prediction.shape[0]
accuracy

result = confusion_matrix(y_test, prediction , normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=result )

disp.plot()

In [ ]:
# cancer evaluation
cancer_iterator = datagen.flow(np.array(np.array(x_test)[np.array(y_test) == 1]), np.array(y_test)[np.array(y_test) == 1])

model.evaluate(cancer_iterator)

In [ ]:
# benign evaluation

benign_iterator = datagen.flow(np.array(np.array(x_test)[np.array(y_test) == 0]), np.array(y_test)[np.array(y_test) == 0])

model.evaluate(benign_iterator)




In [ ]:


## the penalty of missing a cancer is 2 million dollars * .05 100000
## the penalty of diagnosing a cancer when it isn't there is lets say 20000
import keras
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = tf.keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()
plt.figure()
img_size = (50, 50)
path = "imagedata/10253/0/10253_idx5_x1001_y1001_class0.png"
heatmap = make_gradcam_heatmap(datagen.flow(get_img_array(path, img_size))[0], model, "block3_conv3")
plt.matshow(heatmap)
plt.show()

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = load_img(img_path)
    img = tf.keras.utils.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


save_and_display_gradcam(path, heatmap)

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

y_pred_keras = model.predict(np.array(x_test)).ravel()


In [ ]:
from sklearn.metrics import auc
y_pred0 = [y_prediction[i][1] for i in range(0, y_prediction.shape[0])]
fpr_keras, tpr_keras, thresholds_keras = roc_curve(np.array(y_test), y_pred0)


auc_keras = auc(fpr_keras, tpr_keras)
plt.figure()

plt.xlim(0, 1)
plt.ylim(0, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.savefig("ROC.png")

In [ ]:
np.array(x_test).shape


In [ ]:
model.summary()

In [ ]:
model.add_metric()

In [ ]:
!pip install tensorflow-addons